# Complete Fix for Open Notebook CLI Dependencies

This notebook provides the final solution for all dependency issues with the Open Notebook CLI, including the recently discovered `sblpy` dependency.

## The Evolving Problem

After installing the core dependencies (`loguru`, `pydantic`, etc.), we encountered a new error:

```
Traceback (most recent call last):
  File "/app/open_notebook_cli.py", line 22, in <module>
    from open_notebook.domain.notebook import (
  File "/app/open_notebook/domain/notebook.py", line 7, in <module>
    from open_notebook.database.repository import (
  File "/app/open_notebook/database/repository.py", line 6, in <module>
    from sblpy.connection import SurrealSyncConnection
ModuleNotFoundError: No module named 'sblpy'
```

This indicates that we need to install the `sblpy` package, which is used for SurrealDB connections.

## Updated Solution

Let's update our `on_run.sh` script to handle the complete set of dependencies, including database-specific ones.

In [ ]:
#!/bin/bash
# on_run.sh - Complete solution for Open Notebook CLI
# This script ensures all required dependencies are installed

CONTAINER_NAME="assistant-notebook-1"
CLI_PATH="/app/open_notebook_cli.py"

# Function to check and install dependencies
install_dependencies() {
  echo "Installing all required dependencies..."
  
  # Core dependencies
  echo "Installing core dependencies..."
  docker exec $CONTAINER_NAME pip install --upgrade pip
  docker exec $CONTAINER_NAME pip install loguru pydantic langchain langgraph
  
  # Database dependencies
  echo "Installing database dependencies..."
  docker exec $CONTAINER_NAME pip install sblpy surrealdb
  
  # Additional dependencies (yaml, etc.)
  echo "Installing additional dependencies..."
  docker exec $CONTAINER_NAME pip install pyyaml typing_extensions humanize
  
  # Plugin dependencies (streamlit, podcastfy)
  echo "Installing plugin dependencies..."
  docker exec $CONTAINER_NAME pip install streamlit podcastfy
  
  echo "All dependencies installed!"
}

# Display help if no arguments provided
if [ $# -eq 0 ]; then
  echo "Usage: ./on_run.sh <command> [options]"
  echo "Example: ./on_run.sh list-notebooks"
  echo ""
  echo "Special commands:"
  echo "  install    - Install all dependencies"
  echo "  shell      - Open a shell in the container"
  exit 1
fi

# Handle special commands
if [ "$1" = "install" ]; then
  install_dependencies
  exit 0
fi

if [ "$1" = "shell" ]; then
  echo "Opening shell in container..."
  docker exec -it "$CONTAINER_NAME" bash
  exit 0
fi

# Always check for missing dependencies before running a command
if ! docker exec $CONTAINER_NAME python -c "import sblpy" &>/dev/null; then
  echo "Missing dependencies detected. Installing..."
  install_dependencies
fi

# Run the command in the container
echo "Running command: python $CLI_PATH $@"
docker exec -it "$CONTAINER_NAME" python "$CLI_PATH" "$@"

## Complete Installation Script

If you prefer a one-time fix that handles everything, here's a complete installation script to run:

In [ ]:
#!/usr/bin/env python3
"""
complete_fix.py - Complete fix for Open Notebook dependencies
"""

import subprocess
import sys

CONTAINER_NAME = "assistant-notebook-1"
ALL_DEPENDENCIES = [
    # Core dependencies
    "loguru", "pydantic", "langchain", "langgraph",
    # Database dependencies
    "sblpy", "surrealdb", 
    # Additional dependencies
    "pyyaml", "typing_extensions", "humanize",
    # Plugin dependencies
    "streamlit", "podcastfy"
]

def run_in_container(cmd, capture_output=True):
    """Run a command in the Docker container"""
    full_cmd = ["docker", "exec", CONTAINER_NAME] + cmd
    try:
        result = subprocess.run(full_cmd, capture_output=capture_output, text=True, check=True)
        return result.stdout if capture_output else None
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {' '.join(cmd)}")
        if e.stderr:
            print(f"Error output: {e.stderr}")
        return None

def check_container():
    """Check if the container is running"""
    try:
        result = subprocess.run(["docker", "ps", "--format", "{{.Names}}"], 
                                capture_output=True, text=True, check=True)
        return CONTAINER_NAME in result.stdout
    except subprocess.CalledProcessError:
        return False

def install_all_dependencies():
    """Install all required dependencies"""
    print("\n===== Installing All Dependencies =====")
    
    # Upgrade pip first
    print("Upgrading pip...")
    run_in_container(["pip", "install", "--upgrade", "pip"], capture_output=False)
    
    # Install all dependencies
    for dep in ALL_DEPENDENCIES:
        print(f"Installing {dep}...")
        run_in_container(["pip", "install", "--force-reinstall", dep], capture_output=False)
    
    print("\nAll dependencies installed!")

def verify_installations():
    """Verify all dependencies are installed correctly"""
    print("\n===== Verifying Installations =====")
    
    all_good = True
    for dep in ALL_DEPENDENCIES:
        print(f"Checking {dep}...")
        try:
            run_in_container(["python", "-c", f"import {dep}"], capture_output=False)
            print(f"✓ {dep} installed correctly")
        except subprocess.CalledProcessError:
            print(f"✗ Failed to import {dep}!")
            all_good = False
    
    return all_good

def create_wrapper_script():
    """Create the updated wrapper script"""
    print("\n===== Creating Wrapper Script =====")
    
    script_content = """#!/bin/bash
# on_run.sh - Complete solution for Open Notebook CLI
# This script ensures all required dependencies are installed

CONTAINER_NAME="assistant-notebook-1"
CLI_PATH="/app/open_notebook_cli.py"

# Function to check and install dependencies
install_dependencies() {
  echo "Installing all required dependencies..."
  
  # Core dependencies
  echo "Installing core dependencies..."
  docker exec $CONTAINER_NAME pip install --upgrade pip
  docker exec $CONTAINER_NAME pip install loguru pydantic langchain langgraph
  
  # Database dependencies
  echo "Installing database dependencies..."
  docker exec $CONTAINER_NAME pip install sblpy surrealdb
  
  # Additional dependencies (yaml, etc.)
  echo "Installing additional dependencies..."
  docker exec $CONTAINER_NAME pip install pyyaml typing_extensions humanize
  
  # Plugin dependencies (streamlit, podcastfy)
  echo "Installing plugin dependencies..."
  docker exec $CONTAINER_NAME pip install streamlit podcastfy
  
  echo "All dependencies installed!"
}

# Display help if no arguments provided
if [ $# -eq 0 ]; then
  echo "Usage: ./on_run.sh <command> [options]"
  echo "Example: ./on_run.sh list-notebooks"
  echo ""
  echo "Special commands:"
  echo "  install    - Install all dependencies"
  echo "  shell      - Open a shell in the container"
  exit 1
fi

# Handle special commands
if [ "$1" = "install" ]; then
  install_dependencies
  exit 0
fi

if [ "$1" = "shell" ]; then
  echo "Opening shell in container..."
  docker exec -it "$CONTAINER_NAME" bash
  exit 0
fi

# Always check for missing dependencies before running a command
if ! docker exec $CONTAINER_NAME python -c "import sblpy" &>/dev/null; then
  echo "Missing dependencies detected. Installing..."
  install_dependencies
fi

# Run the command in the container
echo "Running command: python $CLI_PATH $@"
docker exec -it "$CONTAINER_NAME" python "$CLI_PATH" "$@"
"""
    
    with open("on_run.sh", "w") as f:
        f.write(script_content)
    
    # Make executable
    subprocess.run(["chmod", "+x", "on_run.sh"])
    
    print("Created wrapper script: on_run.sh")
    print("Usage: ./on_run.sh <command> [options]")

def main():
    print("===== Complete Open Notebook Dependency Fix =====")
    
    # Check if container is running
    if not check_container():
        print(f"Error: Container '{CONTAINER_NAME}' is not running.")
        print("Please start the container and try again.")
        return 1
    
    # Install all dependencies
    install_all_dependencies()
    
    # Verify installations
    if not verify_installations():
        print("\nWARNING: Some dependencies could not be installed correctly.")
    else:
        print("\nAll dependencies installed successfully!")
    
    # Create wrapper script
    create_wrapper_script()
    
    print("\n===== Fix Complete =====")
    print("Try running: ./on_run.sh list-notebooks")
    
    return 0

if __name__ == "__main__":
    sys.exit(main())

## How to Use the Complete Fix

1. Save the `complete_fix.py` script and make it executable:
   ```bash
   chmod +x complete_fix.py
   ```

2. Run the fix script:
   ```bash
   python complete_fix.py
   ```

3. After the script completes, you can use the improved `on_run.sh` script:
   ```bash
   ./on_run.sh list-notebooks
   ```

The script will automatically check for and install missing dependencies before running any command.

## Quick Reference

Special commands in the new wrapper script:

```bash
# Install all dependencies manually
./on_run.sh install

# Open a shell in the container
./on_run.sh shell
```

Normal usage:

```bash
./on_run.sh list-notebooks
./on_run.sh create-notebook "My Research" "Research notes"
```

## If Problems Persist

If you still encounter issues after using the updated script, try:

1. Connecting directly to the container to diagnose the issue:
   ```bash
   docker exec -it assistant-notebook-1 bash
   ```

2. Installing the dependencies manually:
   ```bash
   docker exec -it assistant-notebook-1 pip install --force-reinstall loguru pydantic langchain langgraph
   ```

3. Running the more comprehensive setup script if needed:
   ```bash
   python open_notebook_container_setup.py
   ```

The last option will perform a more thorough diagnosis and fix of the container environment.